In [1]:
	
import numpy as np
import cv2
import math
import requests
import pandas as pd
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm
import urllib.parse as parse
import os
import json
import pytubefix
from pytubefix import YouTube
from pytubefix.exceptions import VideoUnavailable, VideoPrivate
from Katna.video import Video
from Katna.writer import KeyFrameDiskWriter
from transformers import AutoProcessor, AutoModelForCausalLM  
from PIL import Image
import requests
import copy
import supervision as sv
from IPython.display import display
import re

In [2]:
%pwd

'/home1/arpitasa/CSCI544'

In [3]:
video_csv = "./summarized_dense_descriptions_1000_final_1.csv"
df = pd.read_csv(video_csv)

In [4]:
df.head()

,video_id,description,youtube_url,dense_frame_descriptions,bart_summarized_video_captions,t5_finetuned_on_movie_summarized_video_captions
0,nwznKOuZM7w,Two people are seen moving around a kitchen qu...,https://www.youtube.com/watch?v=nwznKOuZM7w,"['a man in a kitchen, standing in front of a w...","The image is taken from a low angle, looking d...","+ $a man in a kitchen, standing in front of a ..."
1,ogQozSI5V8U,We see a hallway with a wooden floor. A dog i...,https://www.youtube.com/watch?v=ogQozSI5V8U,['a narrow hallway with a wooden floor. The ha...,"The image is taken from a low angle, looking d...",a dog standing on its hind legs in a hallway. ...
2,nHE7u40plD0,A woman and a man are sitting on the sidewalk ...,https://www.youtube.com/watch?v=nHE7u40plD0,"[""a group of people sitting on a brick patio. ...",The image shows a group of people gathered aro...,a group of people sitting on a brick patio. Th...
3,69IsHpmRyfk,A young girl is seen sitting in a chair with a...,https://www.youtube.com/watch?v=69IsHpmRyfk,"[""The image is a close-up of a young girl's fa...",A person's hand is visible on the right side o...,a close-up of a young girl with blonde hair. S...
4,D18b2IZpxk0,A woman is shown riding a camel past pyramids ...,https://www.youtube.com/watch?v=D18b2IZpxk0,['a person riding a camel in a desert-like lan...,The image is an abstract background with a gra...,a person riding a camel in a desert-like lands...


In [5]:
vid_list = df['video_id'].to_list()#[:10]

In [6]:
youtube_list = df['youtube_url'].to_list()#[:10]

In [12]:
# vid_list

In [7]:
len(os.listdir("./keyframes"))

978

In [13]:
os.mkdir("./video_folder")
os.mkdir("./keyframes")

FileExistsError: [Errno 17] File exists: './video_folder'

In [16]:

# change the for loop range depending on how many videos you want 
for i in range(len(vid_list)): 

    youtube_file = youtube_list[i] 
    video_file = vid_list[i]
    keyframe_location = os.path.join("./keyframes", video_file)
    if os.path.exists(keyframe_location):
        continue
    else: 
        os.mkdir(keyframe_location)
    print("video "+ str(i) + " = ", video_file)
    file_dir = "./video_folder/" 
    file_path = os.path.join(file_dir, video_file+".mp4")
    try: 
        yt = YouTube(youtube_file) 
        stream = yt.streams.get_highest_resolution()
        stream.download(output_path="./video_folder/", filename=video_file+".mp4")
    except VideoUnavailable:
        print("The video is unavailable.")
    except VideoPrivate:
        print("The video is private.")
    except Exception as e:
        print(f"An error occurred: {e}")




    
    vd = Video()

    video = cv2.VideoCapture(file_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    total_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
    if total_frames == 0 or fps == 0: 
        continue
    duration = math.floor(total_frames / fps)
    
    # Print the results
    # print(f"Frames per second: {fps}")
    # print(f"Total number of frames: {total_frames}")
    # print(f"Video duration (seconds): {duration}")
    
    no_of_frames_to_returned = min(int(duration/10), 15)
    print("number of keyframes = ", no_of_frames_to_returned, "total frames = ", total_frames, "video duration = ", duration)
    
    diskwriter = KeyFrameDiskWriter(location=keyframe_location)
    
    vd.extract_video_keyframes(
       no_of_frames=no_of_frames_to_returned, file_path=file_path,
       writer=diskwriter
    )

    os.remove(file_path)
    

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
finetuned_model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning").to(device)
finetuned_tokenizer = GPT2TokenizerFast.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
finetuned_image_processor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

loading configuration file config.json from cache at /home1/arpitasa/.cache/huggingface/hub/models--nlpconnect--vit-gpt2-image-captioning/snapshots/dc68f91c06a1ba6f15268e5b9c13ae7a7c514084/config.json
Model config VisionEncoderDecoderConfig {
  "_name_or_path": "vit-gpt-pt",
  "architectures": [
    "VisionEncoderDecoderModel"
  ],
  "bos_token_id": 50256,
  "decoder": {
    "_attn_implementation_autoset": false,
    "_name_or_path": "",
    "activation_function": "gelu_new",
    "add_cross_attention": true,
    "architectures": [
      "GPT2LMHeadModel"
    ],
    "attn_pdrop": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 50256,
    "chunk_size_feed_forward": 0,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": 50256,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "embd_pdrop": 0.1,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 50256,
    "exponential_decay_length_penalt

In [9]:
device

'cuda'

In [10]:

def load_image(image_path):
    if os.path.exists(image_path):
        return Image.open(image_path)
    else: 
        print("image does not exist") 
        return None
        
# a function to perform inference
def get_caption(model, image_processor, tokenizer, image_path):
    image = load_image(image_path)
    if image is None: 
        return None 
    # preprocess the image
    img = image_processor(image, return_tensors="pt").to(device)
    # generate the caption (using greedy decoding by default)
    output = model.generate(**img)
    # decode the output
    caption = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    return caption
    
caption_dict = {}
dir = "./keyframes" 
i = 0 
for folder in os.listdir(dir): 
    
    print("video "+ str(i) + " = ", folder) 
    i+=1
    folder_dir = os.path.join(dir, folder)
    if not os.path.isdir(folder_dir): 
        print("skipping") 
        continue
    caption_dict[folder] = ""
    captions = ""
    for file in os.listdir(folder_dir): 
        if file == '.DS_Store':
            print("skipping ds store") 
            continue
        file = os.path.join(folder_dir, file)
        # display(load_image(file))
        # get the caption
        caption = get_caption(finetuned_model, finetuned_image_processor, finetuned_tokenizer, file)
        # print(caption)
        if caption is not None: 
            caption_dict[folder] += caption
            caption_dict[folder]+=". "
    print("done\n") 
    
    # caption_list.append(captions)



The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


video 0 =  qdMjXJTsX94


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


done

video 1 =  GwJ9DmiW8dk
done

video 2 =  hwY9jvSjC9I
done

video 3 =  6lYTHj9vImo
done

video 4 =  LOINRpWGrlw
done

video 5 =  IIAaKRUze8A
done

video 6 =  SIX51vtOOek
done

video 7 =  iM-HdlplAhw
done

video 8 =  8eDJXDetgGE
done

video 9 =  7OcfzMIZsxA
done

video 10 =  SdbxPbXeQJ0
done

video 11 =  1KKVWzCwRSA
done

video 12 =  33SI8z8PovA
done

video 13 =  j0Iv6dmSw0k
done

video 14 =  iDz8nKDpumY
done

video 15 =  8H3Wy96BwHM
done

video 16 =  W07hi9Y37Jg
done

video 17 =  XnUi1SG9jxA
done

video 18 =  FMUaey1tigI
done

video 19 =  grroBKRxAMY
done

video 20 =  lydctNDJZ5o
done

video 21 =  TS-p15iwQPc
done

video 22 =  TgantuXBiJg
done

video 23 =  V5MvrOzQWZo
done

video 24 =  cdiazgshHuc
done

video 25 =  FKAlXR9NkKw
done

video 26 =  fJyxb59mA-A
done

video 27 =  yLS0UuNYXOI
done

video 28 =  UhB739uudDo
done

video 29 =  gGYje5UGLdY
done

video 30 =  4KzG9JBE9V8
done

video 31 =  08uNrWxxHGY
done

video 32 =  5Q8ltS84BBg
done

video 33 =  Pr3iVnKpv0c
done

video 34 =  G

In [11]:
df['baseline_keyframe_descriptions'] = df['video_id'].map(caption_dict)

In [12]:
df.to_csv('baseline_output.csv')